# **<font size=7>Lab 4 Report (Group 29)</font>**

<u>รายชื่อสมาชิกในกลุ่ม:</u>
- นายธนัช ธรนิศตระกูล 63340500025
- นายสิรวิชญ์ ลั้วสมบูรณ์ 63340500066

<hr>

จาก Overall Architecture

![Overall Architecture](https://raw.githubusercontent.com/omzlette/FRA333_LAB_HW/main/LAB4/src/doc/architectures_lab4.jpg)


## **<font size=5>List of Via Points</font>**

List of Via Points จะเป็นไฟล์ .yaml ที่เก็บค่า Via Points ของการวาดตัวอักษรทั้งหมดเอาไว้ รวมไปถึงการเปิด-ปิด Marker ที่จะเอาไว้วาดใน rviz2 ด้วย โดย List of Via Points จะถูกส่งเข้าไปใน Scheduler เพื่อนำค่าตำแหน่งส่งออกไปคำนวณในขั้นตอนถัด ๆ ไป

## **<font size=5>Scheduler</font>**

อย่างที่ได้กล่าวไป Scheduler รับค่าจาก List of Via Points จากนั้นจะทำการส่งค่าตำแหน่งออกไป รวมไปถึงการคำนวณหาค่าเวลาที่ต้องใช้ในการทำ Trajectory Generation ด้วย

ในการคำนวณหาเวลาจะสามารถทำได้โดยการตั้งค่าความเร็วสูงสุด ความเร่งสูงสุด และความกระตุกสูงสุด จากนั้นจะนำมาคำนวณโดยใช้สมการดังนี้

$$ T = \frac{a_{max}}{j_{max}} + \frac{v_{max}}{a_{max}} + \frac{\lVert\Delta{\vec{p}}\rVert}{v_{max}}$$

where;  

$ v_{max} $ is the maximum velocity  
$ a_{max} $ is the maximum acceleration  
$ j_{max} $ is the maximum jerk  
$ \lVert\Delta{\vec{p}}\rVert $ is the relative distance between 2 points

Code:

![Compute Time code from scheduler.py](https://raw.githubusercontent.com/omzlette/FRA333_LAB_HW/main/LAB4/src/doc/codeImg/computeTime.png)

## **<font size=5>Trajectory Generator</font>**

ใน Trajectory Generator จะเป็นการสร้างตำแหน่งและความเร็ว Reference (Reference Position & Reference Velocity) โดยจะใช้ Quintic Trajectory Generator และ Evaluate ตามเวลาออกมาก่อนนำไปเข้า Linear Interpolation

**Quintic Trajectory Generator**

$$ 

\left[\begin{matrix}a_{0}\\a_{1}\\a_{2}\\a_{3}\\a_{4}\\a_{5}\end{matrix}\right] =

\left[\begin{matrix}1 & t_{0} & t_{0}^{2} & t_{0}^{3} & t_{0}^{4} & t_{0}^{5}\\0 & 1 & 2 t_{0} & 3 t_{0}^{2} & 4 t_{0}^{3} & 5 t_{0}^{4}\\0 & 0 & 2 & 6 t_{0} & 12 t_{0}^{2} & 20 t_{0}^{3}\\1 & t_{f} & t_{f}^{2} & t_{f}^{3} & t_{f}^{4} & t_{f}^{5}\\0 & 1 & 2 t_{f} & 3 t_{f}^{2} & 4 t_{f}^{3} & 5 t_{f}^{4}\\0 & 0 & 2 & 6 t_{f} & 12 t_{f}^{2} & 20 t_{f}^{3}\end{matrix}\right]^{-1} 

\left[\begin{matrix}\theta_{0}\\\omega_{0}\\\alpha_{0}\\\theta_{f}\\\omega_{f}\\\alpha_{f}\end{matrix}\right]

$$

where;  

$ a_{n} $ is the $ n^{th} $ coefficient of the quintic polynomial  
$ t_{0} $ is the starting time  
$ t_{f} $ is the final time  
$ \theta_{0} $ is the starting position  
$ \omega_{0} $ is the starting velocity  
$ \alpha_{0} $ is the starting acceleration  
$ \theta_{f} $ is the final position  
$ \omega_{f} $ is the final velocity  
$ \alpha_{f} $ is the final acceleration  

**Quintic Trajectory Evaluator**

$$

\alpha(t) =
a_{0} + a_{1} t + a_{2} t^2 + a_{3} t^3 + a_{4} t^4 + a_{5} t^5

\\~\\

\dot{\alpha}(t) =
a_{1} + 2 a_{2} t + 3 a_{3} t^2 + 4 a_{4} t^3 + 5 a_{5} t^4

$$

where;

$ \alpha(t) $ is the position coefficient of linear interpolation  
$ \dot{\alpha}(t) $ is the velocity coefficient of linear interpolation  

**Linear Interpolation**

$$

p_{r}(t) =
(1 - \alpha(t))\ \cdot \ p_{i} + (\alpha(t))\ \cdot \ p_{f}

\\~\\

\dot{p_{r}}(t) =
\dot{\alpha}(t)\ \cdot \ (p_{f} - p_{i})

$$

where;

$ p_{r}(t) $ is the reference position  
$ \dot{p_{r}}(t) $ is the reference velocity  

Code: 

![Trajectory Generator, Evaluator and Linear Interpolation code from generator.py](https://raw.githubusercontent.com/omzlette/FRA333_LAB_HW/main/LAB4/src/doc/codeImg/generator.png)

## **<font size=5>Inverse Kinematics</font>**

หลังจากคำนวณหาตำแหน่งและความเร็ว Reference เรียบร้อยแล้ว ค่าดังกล่าวจะถูกส่งออกมาเพื่อคำนวณใน Inverse Kinematics เพื่อเปลี่ยนจาก Task States เป็น Joint States เนื่องจากการควบคุมหุ่นยนต์ของเราจะถูกทำบน Joint Mode

โดยการคำนวณ Inverse Kinematics จะถูกแบ่งออกเป็น 2 ส่วนคือการคำนวณตำแหน่งและการคำนวณความเร็ว

**Inverse Position Kinematics ( $\operatorname{IPK}$ )**

$$

q_{1} =
\arctan(\frac{y}{x})

\\~\\

q_{2} =
\arctan(\frac{S_{2}}{C_{2}})

\\~\\

q_{3} =
\arctan(\frac{S_{3}}{C_{3}})


$$

where;

$$

C_{3} =
\frac{- l_{3}^{2} - l_{e}^{2} + x^{2} + y^{2} + \left(- l_{1} + z\right)^{2}}{2 l_{3} l_{e}}

\\~\\

S_{3} =
\sqrt{1 - C_{3}^{2}}

\\~\\

C_{2} =
\left((l_{3} + l_{e} C_{3}) \sqrt{x^{2} + y^{2}} + (l_{e} S_{3})(z - l_{1})\right)

\\~\\

S_{2} =
\left((-l_{e} S_{3}) \sqrt{x^{2} + y^{2}} + (l_{3} + l_{e} C_{3})(z - l_{1})\right)

$$

Code:

![Inverse Position Kinematics code from traject_tracking_server.py](https://raw.githubusercontent.com/omzlette/FRA333_LAB_HW/main/LAB4/src/doc/codeImg/IPK.png)

**Inverse Velocity Kinematics ( $\operatorname{IVK}$ )**

ในการหา $\operatorname{IVK}$ จะต้องทำการหา Angular Jacobian Matrix ก่อน จากนั้นจะสามารถหา Linear Jacobian Matrix ได้ และนำมา Inverse เพื่อนำไปคูณกับความเร็วในแต่ละแกน

<u>Angular Jacobian Matrix</u>

ในการหา Angular Jacobian Matrix จะต้องทำ $\operatorname{IPK}$ มาก่อน เนื่องจากจะต้องใช้ค่า Joint แต่ละ Joint ($q_{1}, q_{2}, q_{3}$) มาคำนวณ นอกจากนั้นยังจะต้องใช้ Position Matrix และ Rotation Matrix ของแต่ละ Joint เทียบกับแกนโลก

$$

r_{1}^{0} =
\left[\begin{matrix}\cos{\left(q_{1} \right)} & - \sin{\left(q_{1} \right)} & 0
\\
\sin{\left(q_{1} \right)} & \cos{\left(q_{1} \right)} & 0
\\
0 & 0 & 1\end{matrix}\right]

\\~\\

r_{2}^{0} =
\left[\begin{matrix}\cos{\left(q_{1} \right)} \cos{\left(q_{2} \right)} & - \sin{\left(q_{2} \right)} \cos{\left(q_{1} \right)} & \sin{\left(q_{1} \right)}
\\
\sin{\left(q_{1} \right)} \cos{\left(q_{2} \right)} & - \sin{\left(q_{1} \right)} \sin{\left(q_{2} \right)} & - \cos{\left(q_{1} \right)}
\\
\sin{\left(q_{2} \right)} & \cos{\left(q_{2} \right)} & 0\end{matrix}\right]

\\~\\

r_{3}^{0} =
\left[\begin{matrix}- \sin{\left(q_{2} \right)} \sin{\left(q_{3} \right)} \cos{\left(q_{1} \right)} + \cos{\left(q_{1} \right)} \cos{\left(q_{2} \right)} \cos{\left(q_{3} \right)} & - \sin{\left(q_{2} \right)} \cos{\left(q_{1} \right)} \cos{\left(q_{3} \right)} - \sin{\left(q_{3} \right)} \cos{\left(q_{1} \right)} \cos{\left(q_{2} \right)} & \sin{\left(q_{1} \right)}
\\
- \sin{\left(q_{1} \right)} \sin{\left(q_{2} \right)} \sin{\left(q_{3} \right)} + \sin{\left(q_{1} \right)} \cos{\left(q_{2} \right)} \cos{\left(q_{3} \right)} & - \sin{\left(q_{1} \right)} \sin{\left(q_{2} \right)} \cos{\left(q_{3} \right)} - \sin{\left(q_{1} \right)} \sin{\left(q_{3} \right)} \cos{\left(q_{2} \right)} & - \cos{\left(q_{1} \right)}
\\
\sin{\left(q_{2} \right)} \cos{\left(q_{3} \right)} + \sin{\left(q_{3} \right)} \cos{\left(q_{2} \right)} & - \sin{\left(q_{2} \right)} \sin{\left(q_{3} \right)} + \cos{\left(q_{2} \right)} \cos{\left(q_{3} \right)} & 0\end{matrix}\right]

\\~\\

r_{z} =
\begin{bmatrix}
0
\\
0
\\
1
\end{bmatrix}

\\~\\

J_{\omega, 0}^{1} =
r_{1}^{0} r_{z}

\\~\\

J_{\omega, 0}^{2} =
r_{2}^{0} r_{z}

\\~\\

J_{\omega, 0}^{3} =
r_{3}^{0} r_{z}

$$

where;

$ r_{n}^{0} $ is the rotation matrix of joint $ n^{th} $ in reference frame 0  
$ r_{z} $ is the rotation matrix of z-axis in reference frame 0  
$ J_{\omega, 0}^{n} $ is the angular jacobian matrix of joint $ n^{th} $ in reference frame 0  


<u>Linear Jacobian Matrix</u>

ในการทำ Linear Jacobian Matrix จะต้องนำ Angular Jacobian Matrix มา Cross Product กับผลต่างของตำแหน่ง Reference ของ End-Effector และตำแหน่ง Reference ของแต่ละ Joint

$$

p_{1}^{0} =
\begin{bmatrix}
0
\\
0
\\
l_{1}
\end{bmatrix}

\\~\\

p_{2}^{0} =
\begin{bmatrix}
0
\\
0
\\
l_{1}
\end{bmatrix}

\\~\\

p_{3}^{0} =
\begin{bmatrix}
l_{3} \cos{\left(q_{1} \right)} \cos{\left(q_{2} \right)}
\\
l_{3} \sin{\left(q_{1} \right)} \cos{\left(q_{2} \right)}
\\
l_{1} + l_{3} \sin{\left(q_{2} \right)}
\end{bmatrix}

\\~\\

p_{e}^{0} =
\begin{bmatrix}
\left(l_{3} \cos{\left(q_{2} \right)} + l_{e} \cos{\left(q_{2} + q_{3} \right)}\right) \cos{\left(q_{1} \right)}
\\
\left(l_{3} \cos{\left(q_{2} \right)} + l_{e} \cos{\left(q_{2} + q_{3} \right)}\right) \sin{\left(q_{1} \right)}
\\
l_{1} + l_{3} \sin{\left(q_{2} \right)} + l_{e} \sin{\left(q_{2} + q_{3} \right)}
\end{bmatrix}

\\~\\

J_{v,0}^{1} =
J_{\omega, 0}^{1} \times (p_{e}^{0} - p_{1}^{0})

\\~\\

J_{v,0}^{2} =
J_{\omega, 0}^{2} \times (p_{e}^{0} - p_{2}^{0})

\\~\\

J_{v,0}^{3} =
J_{\omega, 0}^{3} \times (p_{e}^{0} - p_{3}^{0})

$$

where;

$ p_{n}^{0} $ is the position matrix of joint $ n^{th} $ in reference frame 0  
$ p_{e}^{0} $ is the position matrix of end-effector in reference frame 0  
$ J_{v, 0}^{n} $ is the linear jacobian matrix of joint $ n^{th} $ in reference frame 0  

รวม Linear Jacobian Matrix ของทุก Joint เข้าด้วยกันจะได้

$$

J_{v}^{3}(\vec{q}) =
\left[
    J_{v,0}^{1} \vert J_{v,0}^{2} \vert J_{v,0}^{3}
\right]

$$

Code:

![Angular and Linear Jacobian Matrix code from traject_tracking_server.py](https://raw.githubusercontent.com/omzlette/FRA333_LAB_HW/main/LAB4/src/doc/codeImg/jacobianMat.png)

จากนั้นจะสามารถหา $ \dot{q} $

$$

\dot{q} =
J_{v}^{3}(\vec{q})^{-1}
\begin{bmatrix}
v_{x}
\\
v_{y}
\\
v_{z}
\end{bmatrix}

$$

where;

$ \dot{q} $ is the angular velocity of each joint  

Code:

![Inverse Velocity Kinematics code from traject_tracking_server.py](https://raw.githubusercontent.com/omzlette/FRA333_LAB_HW/main/LAB4/src/doc/codeImg/IVK.png)

## **<font size=5>Trajectory Tracking</font>**

ภายใน Trajectory Tracking จะมีการรับค่าทั้งหมด 3 อย่างคือ Reference Joint States, Joint States ณ ปัจจุบัน และ Enabler (ตัวสั่งการให้ Tracker ทำงาน) และจะมีค่าที่ปล่อยออกไปคือค่า Velocity ที่ได้จาก Control Policy (PI Controller)

Reference Joint States จะถูกส่งออกมาจาก Inverse Kinematics ในหัวข้อก่อนหน้า

Joint States ณ ปัจจุบัน จะถูกส่งออกมาจาก Joint State Broadcaster ซึ่งเป็น Node สำหรับการควบคุมหุ่นยนต์ (Controller Manager)

Enabler จะถูกส่งออกมาจาก Scheduler เมื่อตัวหุ่นยนต์ยังไม่ถึง Via Point จุดหมาย

**ค่า Velocity ที่ได้จาก Control Policy (PI Controller)**

เนื่องจากค่าที่รับเข้ามาเป็นค่า ณ ตำแหน่งปัจจุบัน และค่า Reference Joint States ซึ่งคือจุดต่อไปที่หุ่นยนต์จะต้องไป จึงจำเป็นที่จะต้องมีการคำนวณเพื่อควบคุมความเร็วสำหรับการไปสู่ตำแหน่งต่อไป โดยสมการจะเป็นดังนี้

$$

\pi(t,q) =
\dot{q}_{r}(t) + K_{p} \cdot \left(q_{r}(t) - q\right) + K_{i} \cdot \int_{\tau=0}^{t} \left(q_{r}(\tau) - q\right) d\tau

$$

where;

$K_{p}$ is the proportional gain  
$K_{i}$ is the integral gain

Code:

![PI Controller code from tracker.py](https://raw.githubusercontent.com/omzlette/FRA333_LAB_HW/main/LAB4/src/doc/codeImg/PIControl.png)


## **<font size=5>Proximity Checker</font>**

ในส่วนของ Proximity Checker จะมีการรับค่าทั้งหมด 2 ค่า คือ Joint States ณ ปัจจุบัน และ Via Points ที่ต้องการไป และจะมีค่าที่ปล่อยออกไปคือ Reached Flag (สัญญาณที่บ่งบอกว่าหุ่นยนต์ได้ไปถึง Via Point จุดหมายแล้ว) นอกจากนี้ยังสามารถรับ Argument เพิ่มเติมได้ 1 อย่าง คือ ktype เพื่อใช้เลือกว่าจะทำ Proximity Check ใน Joint Space หรือ Task Space

**การทำ Proximity Check ใน Joint Space**

เริ่มจากการเปลี่ยนค่า Via Point จุดหมาย (ที่อยู่บน Task Space) ให้กลายเป็น Joint Configuration เป้าหมาย (ที่อยู่บน Joint Space) โดยใช้ Inverse Position Kinematics จากหัวข้อก่อนหน้า

จากนั้นนำ Joint Configuration เป้าหมาย มาลบกับ Joint States ณ ปัจจุบัน และนำค่าที่ได้มาหา Norm เพื่อนำมาเทียบกับค่า Threshold ที่กำหนดไว้ ถ้าค่าที่ได้น้อยกว่า Threshold จะทำให้ Reached Flag มีค่าเป็น True (หุ่นยนต์ได้ไปถึง Via Point จุดหมายแล้ว) และถ้ามากกว่า Threshold จะมีค่าเป็น False (หุ่นยนต์ยังไปไม่ถึง Via Point จุดหมาย)
โดยสมการจะเป็นดังนี้

$$

||IPK(p_{f}) - q|| \leq \epsilon

$$

where;  
$p_{f}$ is the final position of the robot  
$q$ is the current joint states  
$\epsilon$ is the threshold

Code:

![IPK Proximity Checker](https://raw.githubusercontent.com/omzlette/FRA333_LAB_HW/main/LAB4/src/doc/codeImg/IPKProximity.png)

**การทำ Proximity Check ใน Task Space**

เริ่มจากการเปลี่ยนค่า Joint States ณ ปัจจุบัน (ที่อยู่บน Joint Space) ให้กลายเป็น Via Point ณ ปัจจุบัน (ที่อยู่บน Task Space) โดยใช้ Forward Position Kinematics
โดยสมการจะเป็นดังนี้

$$

x = \left(l_{3} \cos{\left(q_{2} \right)} + l_{e} \cos{\left(q_{2} + q_{3} \right)}\right) \cos{\left(q_{1} \right)} 

\\~\\

y = \left(l_{3} \cos{\left(q_{2} \right)} + l_{e} \cos{\left(q_{2} + q_{3} \right)}\right) \sin{\left(q_{1} \right)} 

\\~\\

z = l_{1} + l_{3} \sin{\left(q_{2} \right)} + l_{e} \sin{\left(q_{2} + q_{3} \right)}

$$

Code:

![IPK Proximity Checker](https://raw.githubusercontent.com/omzlette/FRA333_LAB_HW/main/LAB4/src/doc/codeImg/FK.png)

จากนั้นนำ Via Point เป้าหมาย มาลบกับ Via Point ณ ปัจจุบัน และนำค่าที่ได้มาหา Norm เพื่อนำมาเทียบกับค่า Threshold ที่กำหนดไว้ ถ้าค่าที่ได้น้อยกว่า Threshold จะทำให้ Reached Flag มีค่าเป็น True (หุ่นยนต์ได้ไปถึง Via Point จุดหมายแล้ว) และถ้ามากกว่า Threshold จะมีค่าเป็น False (หุ่นยนต์ยังไปไม่ถึง Via Point จุดหมาย)
โดยสมการจะเป็นดังนี้

$$

||p_{f} - FPK(q)|| \leq \epsilon

$$

where;  
$p_{f}$ is the final position of the robot  
$q$ is the current joint states  
$\epsilon$ is the threshold

Code:

![IPK Proximity Checker](https://raw.githubusercontent.com/omzlette/FRA333_LAB_HW/main/LAB4/src/doc/codeImg/FPKProximity.png)

